Вот код который сначала не заработал - optimizer.zero_grad() сначала стоял в во втором if (я сделал метку что бы можнобыло найти через crl+f) <1>. Потом я перенес на позицию <2> и все заработало

In [36]:
# in: model, optimizer, loss
def teach_it(model, optimizer, criterion, silent_mod = False):
    max_epochs = 10
    accuracy = {"train": [], "valid": []}
    for epoch in range(max_epochs):
        for k, dataloader in loaders.items():
            epoch_correct = 0
            epoch_all = 0
            for x_batch, y_batch in dataloader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                if k == "train":
                     
                     # <2>
                    optimizer.zero_grad()
                    model.train()
                    #print(device)
                    outp = model(x_batch)
                else:
                     
                    model.eval()
                    with torch.no_grad():
                        outp = model(x_batch)

                preds = outp.argmax(-1)
                correct =  sum(preds == y_batch)  
                all =  y_batch.shape[0]  
                epoch_correct += correct 
                epoch_all += all
                if k == "train":
                    labels = torch.eye(10).to(device)[y_batch]
                    loss = criterion(outp, labels)

                    #optimizer.zero_grad() <1>
                    ##
                    #'''я убрал зероград тут, переместил в первый if и все заработало
                    # до этого в моих моделях все работало нормально '''
                    ##
                    loss.backward()
                    optimizer.step()

            if not silent_mod:
                if k == "train":
                    print(f"Epoch: {epoch+1}")
                print(f"Loader: {k}. Accuracy: {epoch_correct/epoch_all}")
            accuracy[k].append(epoch_correct/epoch_all)

    return accuracy



Однако, когда я обучал mnist и писал полностью свой код я делал в таком порядке:  

    outputs = model(imgs)  
    loss = loss_fn(outputs, labels)  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()  
    
    и все отлично работало

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    log_records = []
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader: 
            imgs = imgs.to(device)
            labels = labels.to(device)
            labels = torch.eye(num_classes).to(device)[labels]#.to(device)
            
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            log_rec1 = '\n{}\nEpoch {}\nTraining loss {}\n'.format(
                datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), epoch,
                loss_train / len(train_loader))

            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), epoch,
                loss_train / len(train_loader)))
            correct, total_size = evaluate_model(model, test_loader)

            log_rec2 = "Accuracy:" + str(correct / total_size) + '\n'
            print(log_rec2)
            log_records.append(log_rec1)
            log_records.append(log_rec2)
    #


Получилось так что из такой формы:  

    outputs = model(imgs)  
    loss = loss_fn(outputs, labels)  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Получилась такая:   
 
    optimizer.zero_grad()  
    outputs = model(imgs)  
    loss = loss_fn(outputs, labels)  
    loss.backward()
    optimizer.step()